In [1]:
import numpy as np
import pandas as pd
import folium

print(folium.__file__)
print(folium.__version__)

from matplotlib import pyplot as plt
%matplotlib inline

/root/.pyenv/versions/knp_hack_env_2.7.11/lib/python2.7/site-packages/folium/__init__.pyc
0.2.1


In [2]:
import sqlite3
import pandas.io.sql as sql
conn = sqlite3.connect('../geo.db')

In [4]:
q = """
SELECT
    id, name, latitude, longitude, geo_hash, pref_code, region_code, source_type
FROM
    GeoCollection
WHERE
    41.433479 <= latitude  AND latitude <= 46.271147
AND 139.367279 <= longitude AND longitude <= 150.974759
"""
#    35.4 <= latitude  AND latitude <= 35.7
#AND 139.3 <= longitude AND longitude <= 140.0
#    35.483928  <= latitude  AND latitude <= 35.668173
#AND 139.308550 <= longitude AND longitude <= 139.993098
df = sql.read_sql(q, conn)
df

,id,name,latitude,longitude,geo_hash,pref_code,region_code,source_type
0,1,Hakodate Station,41.773736,140.726398,xpkjd,01,01202,fb
1,2,函館朝市,41.772549,140.725418,xpkjd,01,01202,fb
2,3,"Hakodate, Hokkaido",41.766700,140.733000,xpkjd,01,01202,fb
3,4,Hakodate Morning Fish Market,41.772355,140.725091,xpkjd,01,01202,fb
4,5,函館海鮮料理海光房,41.772899,140.724401,xpkjd,01,01202,fb
5,6,函館駅,41.773563,140.726582,xpkjd,01,01202,fb
6,7,金森赤レンガ倉庫,41.766627,140.717545,xpkjd,01,01202,fb
7,8,函館山展望台,41.759420,140.704552,xpkj9,01,01202,fb
8,9,ラッキーピエロ函館駅前店,41.771588,140.729160,xpkjd,01,01202,fb
9,10,函館五陵郭城,41.772873,140.726482,xpkjd,01,01202,fb


In [114]:
ids = df['id'].values
lats = df['latitude'].values
lons = df['longitude'].values
popups = df['name'].values
sources = df['source_type'].values

locations = list(zip(lats, lons))

In [115]:
from pyproj import Geod
geod = Geod(ellps='WGS84')
def distance(p1, p2):
    fa, ba, d = geod.inv(p1[0], p1[1], p2[0], p2[1])        
    return d

In [116]:
geo_points = zip(lats, lons, ids, popups, sources)

nayose_points = list()
nayose_names = list()

import itertools

for pair in itertools.combinations(geo_points, 2):
    d = distance((pair[0][1], pair[0][0]), (pair[1][1], pair[1][0]))
    if d < 100.0 and pair[0][3] == pair[1][3]:
        nayose_points.append((pair[0][0], pair[0][1]))
        nayose_points.append((pair[1][0], pair[1][1]))
        nayose_names.append('%s (%s:%s)' % (pair[0][3], pair[0][4], pair[0][2]))
        nayose_names.append('%s (%s:%s)' % (pair[1][3], pair[1][4], pair[1][2]))

In [117]:
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[np.mean(lats), np.mean(lons)],
                  tiles='Cartodb Positron', zoom_start=7.)

mapa.add_children(MarkerCluster(locations=nayose_points, popups=nayose_names))
mapa.save('/root/workspaces/b.html')

In [26]:
from folium.plugins import MarkerCluster

mapa = folium.Map(location=[np.mean(lats), np.mean(lons)],
                  tiles='Cartodb Positron', zoom_start=7.)

mapa.add_children(MarkerCluster(locations=locations, popups=popups))
mapa.save('/root/workspaces/a.html')

In [8]:
#import pdb
#pdb.set_trace()
#% debug --breakpoint folium/map.py:174